<a href="https://colab.research.google.com/github/longqua69/practice-deep-learning/blob/44_tf_data_pipeline/44_tf_data_pipeline_exercise/44_tf_data_pipeline_exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

# Create dataset